# SVM model for toxic YouTube comments

En este notebook entrenamos un modelo de **SVM (LinearSVC)** para detectar
comentarios de odio/toxicidad en YouTube.

Objetivos:

- Cargar el dataset **preprocesado** (texto limpio/lematizado + etiquetas).
- Definir columna de texto y columna objetivo (`IsToxic` u otra).
- Vectorizar el texto con **TF-IDF** (unigramas y bigramas).
- Entrenar un modelo **LinearSVC** con `class_weight="balanced"`.
- Evaluar el modelo (accuracy, precision, recall, f1, ROC-AUC).
- Guardar el modelo entrenado como **`.pkl`** en `models/`.
- Guardar las métricas en un **`.json`** en `results/` con el formato acordado.


## 1. Imports y configuración

Importamos todas las librerías necesarias para:

- Carga de datos y manejo de rutas (`pandas`, `pathlib`).
- Modelado (`scikit-learn`).
- Guardado de modelo (`joblib`) y métricas (`json`).
- Medición de tiempo y timestamp (`datetime`).


In [1]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import json
from datetime import datetime

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


## 2. Carga del dataset preprocesado

Cargamos el CSV **ya preprocesado** generado en el notebook de preprocessing
(`preprocessing_eda.ipynb` o similar).

- Ajusta la ruta `PREPROCESSED_PATH` según el nombre real de tu fichero.
- Este CSV debería contener:
  - Una columna de texto procesado (`text_processed` o similar).
  - La columna objetivo (`IsToxic` u otra).
  - Opcionalmente, features numéricos diseñados a mano (longitud, mayúsculas, etc.).


In [3]:
# Ajusta esta ruta al nombre del CSV que generaste en preprocessing
PREPROCESSED_PATH = Path("../../data/preprocessing_data/youtoxic_english_1000_clean.csv")

df = pd.read_csv(PREPROCESSED_PATH)

print("Shape del dataset preprocesado:", df.shape)
df.head()


Shape del dataset preprocesado: (997, 18)


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsReligiousHate,text_basic,text_classic,text_len_classic,word_count_classic,uppercase_ratio,exclamation_count,hate_words_count
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,If only people would just take a step back and...,people would take step back make case wasnt an...,850,129,0.014121,0,2
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,Law enforcement is not trained to shoot to app...,law enforcement trained shoot apprehend traine...,90,13,0.036232,0,3
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\r\nDont you reckon them 'black lives matter' ...,True,True,False,False,True,False,False,False,Dont you reckon them 'black lives matter' bann...,dont reckon black life matter banner held whit...,252,40,0.002375,0,1
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,There are a very large number of people who do...,large number people like police officer called...,339,49,0.015464,0,0
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,"The Arab dude is absolutely right, he should h...",arab dude absolutely right shot extra time sho...,138,23,0.020576,0,1


## 3. Definición de columnas de texto, target y features numéricas

Nuestro dataset preprocesado incluye:

- `text_classic`: texto preprocesado pensado para **modelos clásicos**
  (TF-IDF, Naive Bayes, Regresión Logística, SVM, etc.).
- `text_basic`: texto más "ligero" para modelos **modernos** (embeddings, transformers, etc.).
- 5 features numéricas de apoyo:
  - `text_len_classic`
  - `word_count_classic`
  - `uppercase_ratio`
  - `exclamation_count`
  - `hate_words_count`

Como SVM es un modelo clásico, en este notebook usaremos **`text_classic`**
junto con esas 5 features numéricas.

También definimos la columna objetivo (`IsToxic`), que representa la tarea de
clasificación binaria (tóxico / no tóxico).


In [5]:
# Texto para modelos clásicos
TEXT_COL = "text_classic"

# Columna objetivo binaria (ajusta si usáis otra, por ejemplo IsAnyToxic)
TARGET_COL = "IsToxic"

# Features numéricas ya creadas en el preprocessing
NUMERIC_FEATURES = [
    "text_len_classic",
    "word_count_classic",
    "uppercase_ratio",
    "exclamation_count",
    "hate_words_count",
]

print("Comprobación de columnas:")
print("  TEXT_COL existe      :", TEXT_COL in df.columns)
print("  TARGET_COL existe    :", TARGET_COL in df.columns)
print("  Numeric features OK  :", [c for c in NUMERIC_FEATURES if c in df.columns])

# Eliminamos filas con target nulo por seguridad
df = df.dropna(subset=[TARGET_COL]).reset_index(drop=True)

X_text = df[TEXT_COL]
y = df[TARGET_COL].astype(int)

n_samples = df.shape[0]
print(f"\nNúmero de muestras utilizadas: {n_samples}")
print("\nDistribución de la clase objetivo:")
print(y.value_counts(normalize=True).sort_index())


Comprobación de columnas:
  TEXT_COL existe      : True
  TARGET_COL existe    : True
  Numeric features OK  : ['text_len_classic', 'word_count_classic', 'uppercase_ratio', 'exclamation_count', 'hate_words_count']

Número de muestras utilizadas: 997

Distribución de la clase objetivo:
IsToxic
0    0.539619
1    0.460381
Name: proportion, dtype: float64


## 4. Train/Test split estratificado

Dividimos el dataset en:

- `X_train_text`, `X_test_text`: textos de train y test.
- `y_train`, `y_test`: etiquetas de train y test.

Usamos estratificación por la columna objetivo para mantener la misma
proporción de tóxico/no tóxico en ambos conjuntos.


In [6]:
TEST_SIZE = 0.2

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y,
)

train_size_real = len(X_train_text) / n_samples
test_size_real = len(X_test_text) / n_samples

print(f"Tamaño train: {len(X_train_text)} ({train_size_real:.4f})")
print(f"Tamaño test: {len(X_test_text)} ({test_size_real:.4f})")


Tamaño train: 797 (0.7994)
Tamaño test: 200 (0.2006)
